In [1]:
import math
import numpy as np
import csv
from operator import itemgetter

def stlGen(file):
    with open(file,'r') as stlfile:
       
        tri = []
        for line in stlfile:
            
            if 'vertex' in line:
                pts = line.replace('vertex','').strip().split(' ')
                
                pts = list(map(float,pts))
                tri.append(pts)
            if len(tri)>2:
                yield tri[:]
                tri = []
           
        return


triGen = stlGen('twist_gear_vase2.stl')
points=list(triGen)


In [2]:
length=len(points)

#triGen = stlGen('twist_gear_vase2.stl')
#points=list(triGen)

def intersect(n,z):
    '''
    find intersecting points
    z is height(layey)
    n is number of points in stl file
    '''
    
    twopoint=[]
 
    p1=points[n][0][2]
    p2=points[n][1][2]
    p3=points[n][2][2]
        
    p_1=np.array(points[n][0])
    p_2=np.array(points[n][1])
    p_3=np.array(points[n][2])
    
    if (p1-z)*(p2-z)<0:
        t=(z-p1)/(p2-p1)
        interpoint1=(p_2-p_1)*t+p_1
        interpoint1=list(interpoint1)
        twopoint.append(interpoint1)
        
    if (p1-z)*(p3-z)<0:
        t=(z-p1)/(p3-p1)   
        interpoint2=(p_3-p_1)*t+p_1
        interpoint2=list(interpoint2)
        twopoint.append(interpoint2)
  
    if (p2-z)*(p3-z)<0:
        t=(z-p2)/(p3-p2)
        interpoint3=(p_3-p_2)*t+p_2
        interpoint3=list(interpoint3)
        twopoint.append(interpoint3)
        
    
    return twopoint


intersect(2800,97.3)

[[-28.054536193029513, -10.634779624664883, 97.3],
 [-35.54328686327078, -13.510309919571041, 97.3]]

In [3]:
def dist(p1,p2):
    
        
    distance=math.sqrt((p1[0]-p2[0])**2+(p1[1]-p2[1])**2+(p1[2]-p2[2])**2)
    return distance






def layers(z):
    val=[]
    pts=[]
    #full length 
    
    for n in range(length):
    #for testing length
    #for n in range(10):
        
       # n is number of set of points
      #  e.g. n=4 -> 4 sets of 2 points
        
        #intersect(n,z)
        if intersect(n,z):
            #a=sorted(intersect(n,z))
            val.append(intersect(n,z))

    #val.sort()
   
    if val:
        pts.append(val[0][0])
        pts.append(val[0][1])
        val.pop(0)
        #a=pts[-1]
        b=[]
   
    
    while len(val)>0:
        a=pts[-1]
        for element in val:
            distance1=math.sqrt((element[0][0]-a[0])**2+(element[0][1]-a[1])**2)
            distance2=math.sqrt((element[1][0]-a[0])**2+(element[1][1]-a[1])**2)
            
            if distance1<0.00001:
                pts.append(element[1])
                #b=element[1]
                val.remove(element)
            if distance2<0.00001:
                pts.append(element[0])
                #b=element[0]
                val.remove(element)

            #a=b
        
           
    
   # print(pts)
    
    return pts
    #return val

#list_1=np.linspace(0,120,length)
list_1=np.linspace(0,100,500)

def ful_list():
    a=[]
    
    for z in list_1:
        l=layers(z)
        a.append(layers(z))
       # print(len(l))
    return a



#list_pass=ful_list()

#len(val)
#val[0]=sorted(val[0])
#val=layers
#print(val[0])
#print(val[0]) -> fist 2 points =ptset
#print(val[0][0]) -> 1st point  =ptset[0]
#print(val[0][1])-> 2nd point   =ptset[1]
#print(val[0][0][0])-> x coord of 1st point  =ptset[0][0]
#for z in range(100):
 #   print(layers(z))

#layers(10)
ful_list1=ful_list()
#print(ful_list1[1])

#print(len(layers(20)))



In [4]:


rate=0.0332
#rate=5.57
e=[]
def extrusion(ful_list1):
    e_layer=[0]
    for layer in ful_list1:
    #for i in range(100):
        for i in range(1,len(layer)):
            a=layer[i-1]
            b=layer[i]
            #print(a)
            #print(b)
            distance=dist(a,b)
            val=rate*distance
            e_layer.append(val)
        e.append(e_layer)
    return e
    

ext1=extrusion(ful_list1)
#print(ext1)


In [5]:
#setcoord = lambda x,y,z,a: f'G1 X{str(x+80)} Y{str(y+80)} Z{str(z)} E{str(a)}\n'
setcoord = lambda x,y,z,e: f'G1 X{str(x+80)} Y{str(y+80)} Z{str(z)} E{str(e)}\n'
def wrcode(pointlis,elist):

    g_code=[]
    e=0
    for i in range(len(pointlis)):
        layer=pointlis[i]
        for j in range(len(layer)):
         #for pt in ptset:
            point=layer[j]
            x=point[0]
            y=point[1]
            z=point[2]
            e+=elist[i][j]
            
            g_code.append(setcoord(x,y,z,e))
        
    
        
   # print(g_code)
    return g_code
    
wrcode1=wrcode(ful_list1,ext1)

#for i in range(len(ext1)):
#    if wrcode1[i]:
#        wrcode1[i]=wrcode1[i]+f' E{str(ext1[i])}\n'
        
#print(wrcode1)
    


In [6]:
def wrt():

    codes=wrcode1
    with open('vase.gcode','w') as f:
        
        
        f.write('M140 S50\n')
        f.write('M105\n')
        f.write('M190 S50\n')
        f.write('M104 S240\n')
        f.write('M105\n')
        f.write('M109 S240\n')
        f.write('M82 ;absolute extrusion mode\n')
        f.write('; Ender 3 Custom Start G-code\n')
        f.write('G92 E0 ; Reset Extruder\n')
        f.write('G28 ; Home all axes\n')
        f.write('G1 Z2.0 F3000 ; Move Z Axis up little to prevent scratching of Heat Bed\n')
        f.write('G1 X0.1 Y20 Z0.3 F5000.0 ; Move to start position\n')
        f.write('G1 X0.1 Y200.0 Z0.3 F1500.0 E15 ; Draw the first line\n')
        f.write('G1 X0.4 Y200.0 Z0.3 F5000.0 ; Move to side a little\n')
        f.write('G1 X0.4 Y20 Z0.3 F1500.0 E30 ; Draw the second line\n')
        f.write('G92 E0 ; Reset Extruder\n')
        f.write('G1 Z2.0 F3000 ; Move Z Axis up little to prevent scratching of Heat Bed\n')
        f.write('G1 X5 Y20 Z0.3 F5000.0 ; Move over to prevent blob squish\n')
        f.write('G1 F1200\n')

        for line in codes:
            f.write(line)
            #f.write('\n')
            
        f.write('G91 ;Relative positioning\n')
        f.write('G1 E-2 F2700 ;Retract a bit\n')
        f.write('G1 E-2 Z0.2 F2400 ;Retract and raise Z\n')
        f.write('G1 X5 Y5 F3000 ;Wipe out\n')
        f.write('G1 Z10 ;Raise Z more\n')
        f.write('G90 ;Absolute positioning\n')

        f.write('G1 X0 Y{machine_depth} ;Present print\n')
        f.write('M106 S0 ;Turn-off fan\n')
        f.write('M104 S0 ;Turn-off hotend\n')
        f.write('M140 S0 ;Turn-off bed\n')

        f.write('M84 X Y E ;Disable all steppers but Z\n')


wrt()